In [ ]:
#unzips datasets

!unzip -o /content/JAFFE-[70,30].zip
!unzip -o /content/CK_dataset.zip


#Imported libraries

import os
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


#loaded images from folder

def load_images_from_folder(folder_path, target_size=(48, 48)):
    images = []
    labels = []

    for label_name in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label_name)

        if os.path.isdir(label_path):
            for filename in os.listdir(label_path):
                if filename.endswith(('.tiff', '.jpg', '.png')):
                    img_path = os.path.join(label_path, filename)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

                    if img is not None:
                        img = cv2.resize(img, target_size)
                        images.append(img)
                        labels.append(label_name)

    return np.array(images), np.array(labels)


# HOG feature extraction

def extract_hog_features(images):
    features = []
    for img in images:
        features.append(
            hog(
                img,
                orientations=9,
                pixels_per_cell=(8, 8),
                cells_per_block=(2, 2),
                block_norm="L2-Hys"
            )
        )
    return np.array(features)


#JAFFE DATASET

# Load JAFFE
jaffe_train_images, jaffe_train_labels = load_images_from_folder(
    "/content/JAFFE-[70,30]/train"
)
jaffe_test_images, jaffe_test_labels = load_images_from_folder(
    "/content/JAFFE-[70,30]/test"
)

print("JAFFE train samples:", len(jaffe_train_images))
print("JAFFE test samples:", len(jaffe_test_images))

# Feature extraction
X_jaffe_train = extract_hog_features(jaffe_train_images)
X_jaffe_test = extract_hog_features(jaffe_test_images)

# Train SVM on JAFFE
svm_jaffe = SVC(kernel="rbf", C=10, gamma="scale")
svm_jaffe.fit(X_jaffe_train, jaffe_train_labels)

# Evaluation
jaffe_train_preds = svm_jaffe.predict(X_jaffe_train)
jaffe_test_preds = svm_jaffe.predict(X_jaffe_test)

print("JAFFE Training Accuracy:",
      accuracy_score(jaffe_train_labels, jaffe_train_preds))
print("JAFFE Testing Accuracy:",
      accuracy_score(jaffe_test_labels, jaffe_test_preds))

# Confusion Matrix
cm_jaffe = confusion_matrix(jaffe_test_labels, jaffe_test_preds)

plt.figure(figsize=(8, 6))
sns.heatmap(
    cm_jaffe,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=np.unique(jaffe_test_labels),
    yticklabels=np.unique(jaffe_test_labels)
)
plt.title("Confusion Matrix – JAFFE Dataset")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# Classification Report
print("JAFFE Classification Report")
print(classification_report(jaffe_test_labels, jaffe_test_preds))


#JAFFE VISUAL EXAMPLE

def show_jaffe_prediction(image_path, model):
    original = cv2.imread(image_path)
    gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)

    resized = cv2.resize(gray, (48, 48))
    features = extract_hog_features([resized])
    prediction = model.predict(features)[0]

    annotated = original.copy()
    cv2.putText(
        annotated,
        prediction,
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.0,
        (255, 0, 0),
        2
    )

    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    plt.title("Predicted Emotion")
    plt.axis("off")
    plt.show()


# Shows 3 JAFFE examples each one with a different emotion
random.seed()
base_path = "/content/JAFFE-[70,30]/test"
shown = 0

for emotion in sorted(os.listdir(base_path)):
    emotion_path = os.path.join(base_path, emotion)
    if os.path.isdir(emotion_path):
        files = [f for f in os.listdir(emotion_path)
                 if f.endswith(('.tiff', '.jpg', '.png'))]
        if files:
            img_path = os.path.join(emotion_path, random.choice(files))
            show_jaffe_prediction(img_path, svm_jaffe)
            shown += 1
        if shown == 3:
            break


#CK dataset

# Load CK
ck_train_images, ck_train_labels = load_images_from_folder(
    "/content/CK_dataset/train"
)
ck_test_images, ck_test_labels = load_images_from_folder(
    "/content/CK_dataset/test"
)

print("CK train samples:", len(ck_train_images))
print("CK test samples:", len(ck_test_images))

# Feature extraction
X_ck_train = extract_hog_features(ck_train_images)
X_ck_test = extract_hog_features(ck_test_images)

# Train SVM on CK
svm_ck = SVC(kernel="rbf", C=10, gamma="scale")
svm_ck.fit(X_ck_train, ck_train_labels)

# Evaluation
ck_train_preds = svm_ck.predict(X_ck_train)
ck_test_preds = svm_ck.predict(X_ck_test)

print("CK Training Accuracy:",
      accuracy_score(ck_train_labels, ck_train_preds))
print("CK Testing Accuracy:",
      accuracy_score(ck_test_labels, ck_test_preds))

# Confusion Matrix
cm_ck = confusion_matrix(ck_test_labels, ck_test_preds)

plt.figure(figsize=(8, 6))
sns.heatmap(
    cm_ck,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=np.unique(ck_test_labels),
    yticklabels=np.unique(ck_test_labels)
)
plt.title("Confusion Matrix – CK Dataset")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# Classification Report
print("CK Classification Report")
print(classification_report(ck_test_labels, ck_test_preds))
